In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/04 22:26:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/04 22:26:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [67]:
# Read the csv files from datasets

#Fake news
df_fake = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/Fake.csv")
# df_fake.show()

In [68]:
#Truth news
df_true = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/True.csv")
#df_true.show()

In [69]:
# count of data and the. umber of coloumns
# print((df_fake.count(), len(df_fake.columns)))
# print((df_true.count(), len(df_true.columns)))

In [70]:
#Add the column for fake news and also a column for true news
from pyspark.sql.functions import lit

df_fake = df_fake.withColumn("Flag", lit("Fake_News"))
#df_fake.show()

In [71]:
df_true = df_true.withColumn("Flag", lit("True_News"))
# df_true.show()

In [72]:
#concat the two tables
df_concat = df_fake.unionByName(df_true, allowMissingColumns=True)
# df_concat.show()

In [74]:
#specifying the row number to each row

from pyspark.sql.functions import monotonically_increasing_id 

df_concat = df_concat.select("*").withColumn("id", monotonically_increasing_id())

df_concat.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- Flag: string (nullable = false)
 |-- id: long (nullable = false)



In [9]:
# print((df_concat.count(), len(df_concat.columns)))

In [10]:
#### has to randomly shuffle the data
import pyspark.sql.functions as F

df_concat = df_concat.select("*").orderBy(F.rand())
# df_concat.show()

In [11]:
# Remove the unwanted coloumn

df_concat =  df_concat.drop(df_concat["date"])
# df_concat

In [12]:
# Remove the title for analysis

df_concat =  df_concat.drop(df_concat["title"])
# df_concat

In [13]:
# In order to avaoid the confusion convert everything to lowercase
from pyspark.sql.functions import col, split, lower, regexp_replace, length

df_concat = df_concat\
.withColumn("text", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.where(length("text")>0)

# df_concat

In [14]:
# remove stop words from it
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df_concat = df_concat.withColumn('words',F.split(F.col('text'),' '))
# df_concat

In [16]:
from pyspark.ml.feature import StopWordsRemover 

stopwordsRemovalFeature = StopWordsRemover(inputCol="words", 
                   outputCol="words without stop").setStopWords(stop)

In [17]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stopwordsRemovalFeature])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df_concat)

In [18]:
df_concat = pipelineFitRemoveStopWords.transform(df_concat)

In [19]:
# df_concat

In [20]:
# convert array of strings to strings
from pyspark.sql.functions import concat_ws

df_concat = df_concat.withColumn("text1", concat_ws(" ", df_concat["words without stop"]))
# df_concat

In [21]:
df_concat = df_concat.drop("words","words without stop")

# df_concat

In [22]:
df_concat = df_concat.drop("text")

# df_concat

In [23]:
df_concat = df_concat.withColumnRenamed("text1","text")

# df_concat

In [24]:
# most frequent words in fake news

# import pyspark.sql.functions as F

# fake_words_freq_df = df_concat.withColumn("text", F.split("text", " ")) \
#     .groupBy("Flag") \
#     .agg(F.collect_list(F.col("text")).alias("text")) \
#     .withColumn("w", F.array_distinct(F.flatten("text"))) \
#     .withColumn("text", F.array_join(F.expr("filter(w, x -> size(filter(text, y -> array_contains(y, x))) = size(text))"), " ")) \
#     .drop("w")

# fake_words_freq_df

In [25]:
# from nltk import tokenize

# token_space = tokenize.WhitespaceTokenizer()

# def counter(text, column_text, quantity):
#     all_words = ' '.join([text for text in text[column_text]])
#     token_phrase = token_space.tokenize(all_words)
#     frequency = nltk.FreqDist(token_phrase)
#     df_frequency = pd.DataFrame({"Word": list(frequency.keys()),
#                                    "Frequency": list(frequency.values())})
#     df_frequency = df_frequency.nlargest(columns = "Frequency", n = quantity)
#     plt.figure(figsize=(12,8))
#     ax = sns.barplot(data = df_frequency, x = "Word", y = "Frequency", color = 'blue')
#     ax.set(ylabel = "Count")
#     plt.xticks(rotation='vertical')
#     plt.show()

In [26]:
# most frequent words in fake news

df_fake = df_concat.filter(df_concat["Flag"] == "Fake_News")

df_fake_text = df_fake.select("text")

# df_fake_text.show()

In [28]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_fake.orderBy(col("count").desc()).show()

+---------+-------+
|     word|  count|
+---------+-------+
|         |1305341|
|    trump|  76773|
|     said|  32978|
|president|  26921|
|   people|  25870|
|      one|  23900|
|    would|  23022|
|  clinton|  18660|
|    obama|  18344|
|     like|  17514|
|   donald|  17160|
|     also|  15029|
|       us|  14363|
|     news|  14179|
|      new|  14095|
|  hillary|  13738|
|     even|  13690|
|     time|  13493|
|    state|  13247|
|    white|  12831|
+---------+-------+
only showing top 20 rows



In [29]:
# most frequent words in true news

df_true = df_concat.filter(df_concat["Flag"] == "True_News")

df_true_text = df_true.select("text")

# df_true_text.show()

In [30]:
df_true_text = df_true_text\
.withColumn("line", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.withColumn("words", split(col("line"), " "))\
.where(length("line")>0)

# df_true_text

In [31]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

# df_freq_fake.orderBy(col("count").desc()).show()

In [75]:
df3.show(200)

+--------------------+----+------------+--------------------+
|             subject|Flag|          id|                text|
+--------------------+----+------------+--------------------+
|           left-news|   0| 51539608306|wonder latino ame...|
|           worldnews|   1|120259084538|rome  reuters    ...|
|        politicsNews|   1| 94489280901|washington  reute...|
|        politicsNews|   1| 94489282582|washington  reute...|
|            politics|   0| 25769804623|president trump f...|
|                News|   0|  8589936987|fox news host fou...|
|           left-news|   0| 42949673879|silly  silly donn...|
|           worldnews|   1|111669152472|madrid  reuters  ...|
|                News|   0| 17179871539|new jersey teenag...|
|                News|   0| 17179870762|group house repub...|
|           worldnews|   1|128849020006|bangkok  reuters ...|
|            politics|   0| 25769805299|msnbc lawrence do...|
|           left-news|   0| 51539608051|would ellen motiv...|
|       

#### Logistic Regression

In [32]:
df = df_concat

# df

In [33]:
from pyspark.sql.functions import when

df3 = df.withColumn("Flag", when(df.Flag == "True_News",1) \
      .when(df.Flag == "Fake_News",0) \
      .otherwise(df.Flag))

# df3

In [35]:
df3 = df3.withColumn("Flag",col("Flag").cast("int"))
df3.printSchema()
df3.count()

root
 |-- subject: string (nullable = true)
 |-- Flag: integer (nullable = true)
 |-- id: long (nullable = false)
 |-- text: string (nullable = false)



44898

In [36]:
# x values and Y values

x = df3.select('text').rdd.flatMap(lambda x: x).collect()
y = df3.select('Flag').rdd.flatMap(lambda x: x).collect()

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.25)

In [38]:
x_train[0]

'washington  reuters    republican u  president elect donald trump removed new jersey governor chris christie leader transition team  handing reins vice president elect mike pence  former u  lawmaker deep washington ties  new york times reported friday   times  citing several sources close transition team  said trump told advisers wanted use pence contacts move transition process along  nbc news also reported handover  came one week two former christie associates found guilty new jersey  bridgegate  scandal   '

In [39]:
y_train[0]

1

In [40]:
#vectorize the text

from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [42]:
xv_train

<33673x105679 sparse matrix of type '<class 'numpy.float64'>'
	with 5475333 stored elements in Compressed Sparse Row format>

In [43]:
xv_train.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [44]:
from sklearn.linear_model import LogisticRegression

In [45]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [46]:
LR.score(xv_test, y_test)

0.9841425389755011

In [47]:
pred_LR = LR.predict(xv_test)
pred_LR

array([1, 1, 1, ..., 1, 1, 1])

In [48]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      5885
           1       0.98      0.99      0.98      5340

    accuracy                           0.98     11225
   macro avg       0.98      0.98      0.98     11225
weighted avg       0.98      0.98      0.98     11225



In [49]:
import re, string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [50]:
import pandas as pd
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    print("*****")
    print(new_xv_test)
    print(type(pred_LR[0]))
    print(type(pred_LR[0]))
    # pred_DT = DT.predict(new_xv_test)
    # pred_GBC = GBC.predict(new_xv_test)
    # pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \n".format(output_lable(pred_LR[0])))

In [52]:
news = str(input())
manual_testing(news)

 LONDON (Reuters) - LexisNexis, a provider of legal, regulatory and business information, said on Tuesday it had withdrawn two products from the Chinese market in March this year after it was asked to remove some content.  The issue of academic freedom in China hit the headlines this week after the leading British academic publisher,  Cambridge University Press, said it had complied with a request to block online access to some scholarly articles in China.  It later reversed its position.   Earlier this year LexisNexis Business Insight Solutions in China was asked to remove some content from its database,  LexisNexis said in a statement.  In March 2017, the company withdrew two products (Nexis and LexisNexis Academic) from the Chinese market.  LexisNexis is owned by information group Relx.


*****
  (0, 104261)	0.062113217743404024
  (0, 102875)	0.08884373683939102
  (0, 102873)	0.09400165271267218
  (0, 101706)	0.03666925169281988
  (0, 98195)	0.05458166324095393
  (0, 96929)	0.06307502582305753
  (0, 96442)	0.039818455021442156
  (0, 89389)	0.039032819256168645
  (0, 87947)	0.08206146553502687
  (0, 83633)	0.1184982886307607
  (0, 82450)	0.05685332712766565
  (0, 80126)	0.08602251020559346
  (0, 80066)	0.02482659723026979
  (0, 79597)	0.05718615515651515
  (0, 79172)	0.1386950791720891
  (0, 78764)	0.07817873577256551
  (0, 75680)	0.1008427948247376
  (0, 75442)	0.09444383072265919
  (0, 74998)	0.1480535164968732
  (0, 74607)	0.049535052730624016
  (0, 73861)	0.05314917160400599
  (0, 69943)	0.06807439451240366
  (0, 68671)	0.06304618034404079
  (0, 65939)	0.15420553110690025
  (0, 59321)	0.12084414939648837
  :	:
  (0, 55599)	0.050652401115840545
  (0, 55397)	0.05642559211190205
  (0, 55067)	0.04633053030584754
  (0, 49261)	0.047025710870372474
  (0, 48133)	0.0971538201

In [63]:
df3.printSchema()

root
 |-- subject: string (nullable = true)
 |-- Flag: integer (nullable = true)
 |-- id: long (nullable = false)
 |-- text: string (nullable = false)



In [64]:
df3.groupby('flag').count()

flag,count
0,23481
1,21417


In [66]:
df3.show(200)

+--------------------+----+------------+--------------------+
|             subject|Flag|          id|                text|
+--------------------+----+------------+--------------------+
|           left-news|   0| 51539608306|wonder latino ame...|
|           worldnews|   1|120259084538|rome  reuters    ...|
|        politicsNews|   1| 94489280901|washington  reute...|
|        politicsNews|   1| 94489282582|washington  reute...|
|            politics|   0| 25769804623|president trump f...|
|                News|   0|  8589936987|fox news host fou...|
|           left-news|   0| 42949673879|silly  silly donn...|
|           worldnews|   1|111669152472|madrid  reuters  ...|
|                News|   0| 17179871539|new jersey teenag...|
|                News|   0| 17179870762|group house repub...|
|           worldnews|   1|128849020006|bangkok  reuters ...|
|            politics|   0| 25769805299|msnbc lawrence do...|
|           left-news|   0| 51539608051|would ellen motiv...|
|       